In [1]:
from multimodemodel import laplacian_mixing_u, _numba_2D_grid_iterator, _cyclic_shift, State, Parameters, Variable
import numpy as np
from tqdm import tqdm as progressbar

In [2]:
@_numba_2D_grid_iterator
def _laplacian_mixing_u_2D(
    i: int,
    j: int,
    ni: int,
    nj: int,
    u: np.ndarray,
    mask_u: np.ndarray,
    mask_q: np.ndarray,
    dx_u: np.ndarray,
    dy_u: np.ndarray,
    dx_q: np.ndarray,
    dy_q: np.ndarray,
    dx_eta: np.ndarray,
    dy_eta: np.ndarray,
    lbc: int,
    a_h: float,
) -> float:  # pragma: no cover
    """Compute laplacian diffusion of u."""
    ip1 = _cyclic_shift(i, ni, 1)
    im1 = _cyclic_shift(i, ni, -1)
    jp1 = _cyclic_shift(j, nj, 1)
    jm1 = _cyclic_shift(j, nj, -1)

    if mask_q[0, j, i] == 0:
        lbc_j = lbc
    else:
        lbc_j = 1

    if mask_q[0, jp1, i] == 0:
        lbc_jp1 = lbc
    else:
        lbc_jp1 = 1

    return (
        a_h
        * mask_u[0, j, i]
        * (
            (dy_eta[j, i] / dx_eta[j, i])
            * (mask_u[0, j, ip1] * u[0, j, ip1] - mask_u[0, j, i] * u[0, j, i])
            - (dy_eta[j, im1] / dx_eta[j, im1])
            * (mask_u[0, j, i] * u[0, j, i] - mask_u[0, j, im1] * u[0, j, im1])
            + (dx_q[jp1, i] / dy_q[jp1, i])
            * lbc_jp1
            * (mask_u[0, jp1, i] * u[0, jp1, i] - mask_u[0, j, i] * u[0, j, i])
            - (dx_q[j, i] / dx_q[j, i])
            * lbc_j
            * (mask_u[0, j, i] * u[0, j, i] - mask_u[0, jm1, i] * u[0, jm1, i])
        )
        / dx_u[j, i]
        / dy_u[j, i]
    )

def laplacian_mixing_u_2D(state: State, params: Parameters) -> State:
    """Compute laplacian diffusion of zonal velocities."""
    grid = state.variables["u"].grid
    lbc = 2 * params.no_slip
    args = (
        grid.shape[grid.dim_x],
        grid.shape[grid.dim_y],
        state.variables["u"].safe_data,
        state.variables["u"].grid.mask,
        state.variables["q"].grid.mask,
        state.variables["eta"].grid.dx,
        state.variables["eta"].grid.dy,
        state.variables["u"].grid.dx,
        state.variables["u"].grid.dy,
        state.variables["v"].grid.dx,
        state.variables["v"].grid.dy,
        lbc,
        params.a_h,
    )
    return State(
        u=Variable(
            _laplacian_mixing_u_2D(*args),
            grid,
            state.variables["u"].time,
        )
    )

In [3]:
from  multimodemodel import StaggeredGrid

c_grid = StaggeredGrid.regular_lat_lon_c_grid(
    lon_start=-50.0,
    lon_end=50.0,
    lat_start=-5.0,
    lat_end=5.0,
    nx=100 * 4 + 1,
    ny=10 * 4 + 1,
    z = np.array([0])
)

In [4]:
from multimodemodel import Parameters, f_on_sphere
params = Parameters(
    coriolis_func=f_on_sphere(omega=7.272205e-05),
    on_grid=c_grid,
    H=np.array([1e3]),
    gamma = np.array([1e-3]),
)

In [5]:
eta = np.expand_dims(np.exp(-((c_grid.eta.x) ** 2 + (c_grid.eta.y - 2.5) ** 2) / 10. ** 2), 0)

In [6]:
t0 = np.datetime64("2001-01-01 00:00")

In [13]:
from multimodemodel import integrate, adams_bashforth3

model_run_2D = integrate(
    State(
        u=Variable(None, c_grid.u, t0),
        v=Variable(None, c_grid.v, t0),
        eta=Variable(eta, c_grid.eta, t0),
        q=Variable(None, c_grid.q, t0)
    ),
    params,
    RHS=laplacian_mixing_u_2D,
    scheme=adams_bashforth3,
    step=0.05,
    time=1.0,
)

model_run_3D = integrate(
    State(
        u=Variable(None, c_grid.u, t0),
        v=Variable(None, c_grid.v, t0),
        eta=Variable(eta, c_grid.eta, t0),
        q=Variable(None, c_grid.q, t0)
    ),
    params,
    RHS=laplacian_mixing_u,
    scheme=adams_bashforth3,
    step=0.05,
    time=1.0,
)

In [14]:
%%timeit -r 10 -n 1000
for i, next_state in enumerate(model_run_3D):
    pass

The slowest run took 382.49 times longer than the fastest. This could mean that an intermediate result is being cached.
3.96 µs ± 11.6 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)


In [15]:
%%timeit -r 10 -n 1000
for i, next_state_2D in enumerate(model_run_2D):
    pass

The slowest run took 303.33 times longer than the fastest. This could mean that an intermediate result is being cached.
3.61 µs ± 10.5 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)
